# Homework 3 - What is the best anime in the world?

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import os
import pickle
import numpy as np
import time
import re
import datetime as dt 
import csv
import pandas as pd

## 1. Data collection

### 1.1. Get the list of animes

In [2]:
def take_n_urls(n):

    main_url = "https://myanimelist.net/topanime.php"

    # this list will contain all the urls we'll retrieve
    urls = [] 

    # each page shows 50 elements and we can retrieve each page by manipulating the "limit" query
    for limit in range(0, n, 50): 
        content = requests.get(main_url,
                               params={"limit": limit})
        if content.status_code == 404:
            print(f"Page with limit {limit} was not found. Interrumpting and returning pages found")
            break
        soup = bs(content.content, "html.parser")

        # from the content of each page we retrieve the portions that contain the urls
        results = soup.find_all("a", 
                                class_= "hoverinfo_trigger fl-l ml12 mr8")

        # finally, we take the string containing each url by taking the attribute href,
        # and we append them in the urls list
        for result in results:
            url = result["href"]
            if url not in urls:  # check for duplicates
                urls.append(url)

    return urls

In [3]:
if "urls.txt" not in os.listdir():
    urls = take_n_urls(20000)
    # Since the output of this step has to be a txt file, here we write one with each
    # url separated by a newline
    with open("urls.txt", "w") as file:
        file.write("\n".join(urls_str))
else:
    with open("urls.txt", "r", encoding="utf8") as file:
        print("Loading urls...")
        urls = file.read().split("\n")
        print("Done!")

Loading urls...
Done!


In [4]:
# we end up with 19131 urls. 
# I added a check that tells us when we have exceeded the length of the ranking list and returns what has been found
# up until that moment (so to avoid losing any more time with get requests that point to nothing)
# I know in the assignment they said 20000 but I'm fairly sure that's all the entries. 
# This is easy to see if we manually set the limit in the url and check the results. 
# For example: https://myanimelist.net/topanime.php?limit=15000 contains rankings 15001-15050. The first entry is
# Big X Episode 0. If we check our list with urls[15000] (remember that our list is 0-indexed) we obtain the same result.
# This to me seems to point to a correct behavior from the function, but let me know what you think.

print(len(urls)) 
print(urls[15000])
urls_str = list(map(str, urls))

19130
https://myanimelist.net/anime/30839/Big_X_Episode_0


### 1.2. Crawl animes

Attention: the index of article start with 0 and not 1 so all ranks are shifted by 1 position

In [27]:
# Here we create the directory where the html pages will be stored
if "html_pages" not in os.listdir():
    os.mkdir("html_pages")

In [30]:
def save_html_pages(urls):
    if "counter_pages" not in os.listdir():
        start = 0
    else:
        with open("counter_pages", "rb") as counter_file:
            start = pickle.load(counter_file) + 1

    print(f"Starting from anime #{start}")
    n = len(urls)
    for i in range(start, n):
        ranking_page = str(int(np.floor(i/50)))
        if i % 50 == 0 or f"ranking_page_{ranking_page}" not in os.listdir("./html_pages"):
            os.mkdir(f"html_pages/ranking_page_{ranking_page}")
        html_page = requests.get(urls[i])
        sleep_modifier = 0
        while html_page.status_code != 200: # if the status_code is not 200, we've exceeded the number of requests and have to wait
            sleep_timer = 20+sleep_modifier
            print(f"Exceeded number of requests while retrieving page #{i}.\nWaiting {sleep_timer} seconds")
            html_page.close()
            time.sleep(sleep_timer)
            html_page = requests.get(urls[i])
            sleep_modifier += 1
        with open (f"html_pages/ranking_page_{ranking_page}/article_{i}.html", "w", encoding="utf-8") as file:
            file.write(html_page.text)
        with open ("counter_pages", "wb") as counter_file:
            pickle.dump(i, counter_file)


In [31]:
save_html_pages(urls)

Starting from anime #175
Exceeded number of requests while retrieving page #175.
Waiting 5 seconds
Exceeded number of requests while retrieving page #175.
Waiting 6 seconds
Exceeded number of requests while retrieving page #175.
Waiting 7 seconds
Exceeded number of requests while retrieving page #175.
Waiting 8 seconds
Exceeded number of requests while retrieving page #326.
Waiting 5 seconds
Exceeded number of requests while retrieving page #326.
Waiting 6 seconds
Exceeded number of requests while retrieving page #326.
Waiting 7 seconds
Exceeded number of requests while retrieving page #326.
Waiting 8 seconds
Exceeded number of requests while retrieving page #326.
Waiting 9 seconds
Exceeded number of requests while retrieving page #326.
Waiting 10 seconds
Exceeded number of requests while retrieving page #326.
Waiting 11 seconds
Exceeded number of requests while retrieving page #326.
Waiting 12 seconds
Exceeded number of requests while retrieving page #326.
Waiting 13 seconds
Exceeded 

KeyboardInterrupt: 

### 1.3 Parse downloaded pages

At this point, you should have all the html documents about the animes of interest and you can start to extract the animes informations. The list of information we desire for each anime and their format is the following:

1. Anime Name (to save as `animeTitle`): String
2. Anime Type (to save as `animeType`): String
3. Number of episode (to save as `animeNumEpisode`): Integer
4. Release and End Dates of anime (to save as `releaseDate` and `endDate`): Convert both release and end date into datetime format.
5. Number of members (to save as `animeNumMembers`): Integer
6. Score (to save as `animeScore`): Float
7. Users (to save as `animeUsers`): Integer
8. Rank (to save as `animeRank`): Integer
9. Popularity (to save as `animePopularity`): Integer
10. Synopsis (to save as `animeDescription`): String
11. Related Anime (to save as `animeRelated`): Extract all the related animes, but only keep unique values and those that have a hyperlink associated to them. List of strings.
12. Characters (to save as `animeCharacters`): List of strings.
13. Voices (to save as `animeVoices`): List of strings
14. Staff (to save as `animeStaff`): Include the staff name and their responsibility/task in a list of lists.


In [316]:
# Here we create the directory where the tsv files will be stored
os.mkdir("tsv_files")

In [387]:
def collect_info(num_article, folder='tsv_files'):
    ranking_page = str(int(np.floor(num_article/50)))
    article=f'html_pages/ranking_page_{ranking_page}/article_{num_article}.html'
    with open(article, "r", encoding="utf-8") as file:
        art= bs(file.read(), 'html.parser')
    
    #animeTitle
    animeTitle = art.find('h1', {'class':"title-name h1_bold_none"}).string
    #print('animeTitle :',animeTitle)
    
    
    #animeType
    animeType = art.find('span', {'class':"information type"}).string
    #print('animeType :',animeType)
    
    
    #animeNumEpisode and Dates (there is not specific name for those two info)
    #list lines with tag <div class="spaceit_pad">
    lines = art.find_all('div', {'class':"spaceit_pad"})
    for line in lines :
        #for each div tag there is one span, so here we look for the span tag with 'Episodes:' and 'Aired'
        sp= line.find('span', {'class':"dark_text"})
        # to avoid error if there is no span
        if sp is not None :
            #for span 'Episodes' (and the div tag which corresponds)
            if sp.string == 'Episodes:' :
                #extract the content of the right div tag and take the third line which correspond to the number of episodes
                animeNumEpisode = int(line.contents[2])
                #animeNumEpisode = int(re.findall(r'-?\d+\.?\d*', str(line))[0])           #if we want to use regex  
            #for span 'Aired' (and the div tag which corresponds)
            if sp.string == 'Aired:' :
                str_dates = line.contents[2].split('\n  ')[1]
                #if "Status: Finished Airing" (there is a endDate)
                if 'to' in str_dates :
                    #extract the content of the right div tag and take the third line which correspond to the dates (fix the issue of '\n')
                    str_releaseDate, str_endDate = str_dates.split(' to ')
                    #convert into a datetime
                    date_format = "%b %d, %Y"
                    releaseDate, endDate = dt.datetime.strptime(str_releaseDate, date_format), dt.datetime.strptime(str_endDate, date_format)
                else :
                    str_releaseDate = str_dates
                    #convert into a datetime
                    date_format = "%b %d, %Y"
                    releaseDate = dt.datetime.strptime(str_releaseDate, date_format)
                    endDate=''
    #print('animeNumEpisode :',animeNumEpisode)
    #print('releaseDate :',releaseDate)
    #print('endDate :',endDate)
    
    
    #animeNumMembers
    animeNumMembers = int(art.find('span', {'class':"numbers members"}).contents[1].string.replace(',',''))
    #print('animeNumMembers :',animeNumMembers)
    
    
    #animeScore
    animeScore = float(art.find('div', {'class':"score-label"}).string)
    #print('animeScore :',animeScore)
    
    
    #animeUsers
    animeUsers = int(art.find('span', {'itemprop':"ratingCount"}).string)
    #print('animeUsers :',animeUsers)
    
    
    #animeRank
    animeRank = int(art.find('span', {'class':"numbers ranked"}).contents[1].string.replace('#',''))
    #print('animeRank :',animeRank)
    
    
    #animePopularity
    animePopularity = int(art.find('span', {'class':"numbers popularity"}).contents[1].string.replace('#',''))
    #print('animePopularity :',animePopularity)
    
    
    #animeDescription
    desc = art.find('p', {'itemprop':"description"}).contents
    animeDescription=''
    #remove <br/> Tag and '\n'
    for ele in desc :
        #delete tags with regex 
        ele = re.sub(re.compile('<.*?>'),'', str(ele))
        animeDescription += ele
    #print('animeDescription :',animeDescription.replace('\n',''))
    
    
    #animeRelated
    animeRelated = []
    #store the table which contain related animes
    table = art.find('table', {'class':"anime_detail_related_anime"})
    if table is not None :
        #store all links/anime related with 'a' Tag
        links = table.find_all('a')
        for link in links :
            # check if there is a hyperlink and add it in the list if yes 
            if (link.get('href') is not None) and (link.string is not None):
                animeRelated += [link.string]
        animeRelated=list(set(animeRelated))
    else :
        animeRelated=''
    #print('animeRelated :',animeRelated)

    
    #animeCharacters
    animeCharacters = art.find_all('h3', {'class':"h3_characters_voice_actors"})
    animeCharacters = [char.string for char in animeCharacters]
    #print('animeCharacters :',animeCharacters)
    
    
    #animeVoices
    td_Voices = art.find_all('td', {'class':"va-t ar pl4 pr4"})
    animeVoices = [voice.find('a').string for voice in td_Voices]
    #print('animeVoices :',animeVoices)
    
    
    #animeStaff
    #if there is a staff, the div which correspond to the table Staff is the second one (there are div with {'class':"detail-characters-list clearfix"})
    if len(art.find_all('div', {'class':"detail-characters-list clearfix"}))>1 :
        div_staff = art.find_all('div', {'class':"detail-characters-list clearfix"})[1] 
        td_staff = div_staff.find_all('td', {'class':"borderClass"})
        animeStaff=[]
        for td in td_staff :
            if td.get('width') == None:
                animeStaff.append([td.find('a').string,td.find('small').string])
    #if there is not staff
    else :
        animeStaff = ''
    #print('animeStaff :',animeStaff)
    
    #create a .tsv file with attributes
    with open(f'{folder}/anime_{num_article}', 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow([animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, \
                            animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, \
                            animeCharacters, animeVoices, animeStaff])
    

In [388]:
for i in range(len(urls)):
    collect_info(i)


ValueError: time data '?' does not match format '%b %d, %Y'

In [389]:
i

31

## 2. Search Engine

### 2.0. Pre-processing

### 2.1. Conjunctive query

## 5. Algorithmic question
You consult for a personal trainer who has a *back-to-back sequence* of requests for appointments. A sequence of requests is of the form
    > 30, 40, 25, 50, 30, 20
where each number is the time that the person who makes the appointment wants to spend.
You need to accept some requests, however you need a break between them, so you cannot accept two consecutive requests. For example, `[30, 50, 20]` is an acceptable solution (of duration *100*), but `[30, 40, 50, 20]` is not, because *30* and *40* are two consecutive appointments. Your goal is to provide to the personal trainer a schedule that maximizes the total length of the accepted appointments. For example, in the previous instance, the optimal solution is `[40, 50, 20]`, of total duration *110*.
1. Write an algorithm that computes the acceptable solution with the longest possible duration.
2. Implement a program that given in input an instance in the form given above, gives the optimal solution.


In [30]:
import numpy as np

### 1. Write an algorithm that computes the acceptable solution with the longest possible duration.

##### Here we consider that all values in the instance are unique and len(instance) = n

To compute the acceptable solution with the longest possible duration, we have to follow several steps :
1. Compute every possible solution : So for that, list all sublists which represent each possible list of appointments.
2. For each sublist, tell if it is acceptable or not, so if there are two consecutive appointments or not.
3. Compute the total duration of each acceptable solution.
4. Finally, return the solution which correspond to the maximum duration.

```
Input: 
    instance: list of length n

function optimal_solution(instance):
    n=len(instance)
    for i=0 to n: 
        sublists = sublists + [all sublists with i elements]
    
    acceptable_solutions=[all element of sublists which are acceptable]
    
    durations = [duration of each element of acceptable_solutions]
    max_duration = max(durations)
    optimal_solutions = [sublists of instance with total duration == max_durations]
    
    return optimal_solutions, max_duration
```

### 2. Implement a program that given in input an instance in the form given above, gives the optimal solution.

First of all, we create a function ```is_acceptable(solution, instance)``` that says if a solution is acceptable or not, i.e. there is not two consecutive requests of the instance in the solution.

Then, the ```longest_acceptable_duration(instance)``` compute all possible sublists of the instance, test it with the function ```is_acceptable(solution, instance)```, sum every acceptable list (to compute the duration of each acceptable solution) and return the maximum. 

In [142]:
def is_acceptable(solution, instance):
    res=True 
    for x,y in zip(solution[:-1],solution[1:]):
        i= instance.index(x)
        #index1 = np.where(np.array(instance) == x)
        #for i in index[0]:
        #if the next element is y 
        if instance[i+1] == y:
            res = False
    return res

def optimal_solution(instance):
    sublists=[]
    
    for i in range(1, len(instance)+1):
        sublists+=[list(x) for x in combinations(instance, i)]
        
    mask = [is_acceptable(solution, instance) for solution in sublists]
    acceptable_sol = np.array(sublists)[mask]
    
    durations = [sum(L) for L in acceptable_sol]
    max_duration = max(durations)
    
    index_optimal_solutions = np.where(np.array(durations)==max_duration)
    
    return list(np.array(acceptable_sol)[index_optimal_solutions]), max_duration

In [143]:
instance = [30, 40, 25, 50, 30, 20]
optimal_solutions, max_duration = optimal_solution(instance)
print(optimal_solutions, max_duration)

[[40, 50, 20]] 110
